In [33]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import math
from tqdm import tqdm
import plotly.graph_objects as go
import plotly.express as px
from functions import *
from tickers_list import *
sns.set_style('whitegrid')

In [2]:
## HARDCODE VALUES

stocks = ["LUPIN.NS", "VOLTAS.NS", "YESBANK.NS"]

INITIAL_AMT_INVESTED = 10000
NUM_OF_SIMULATIONS = 5000
start_date='2023-01-01'
end_date='2023-12-31'

In [28]:
## GATHER STOCK PRICES

stock_prices, stock_dividends = fetch_data(stocks, start_date='2023-01-01', end_date='2023-12-31')

## CALCULATE LOG RETURNS

log_returns = np.log(stock_prices/stock_prices.shift(1))[1:]

## DOWNSIDE DEVIATION (SORTINO RATIO)

log_returns_down_only = log_returns.applymap(lambda x: 0 if x>0 else x)

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:02<00:00,  1.04it/s]


In [93]:
def plot_price(price_data, dividend_data=None, stock_name=None, figure=None):
    
    """Plotly graph, make sure you use plotly elements"""
    
    if figure is None:
        fig = go.Figure()
    else:
        fig = figure
    
    if stock_name is None:
        stock = price_data.name
    else:
        stock = stock_name
    
    ## PLOTTING THE PRICES
    fig.add_trace(go.Scatter(x=price_data.index, y=price_data, mode='lines', name=stock))
    
    if dividend_data is not None:
        ## PLOTTING THE DIVIDEND ROLLOUT
        dividend_rollout_days = dividend_data[dividend_data != 0].index
        price_when_dividend = price_data.loc[dividend_rollout_days]
        dividend_amounts = dividend_data.loc[dividend_rollout_days]
        
        # Constructing hover text without date
        hover_text = [f"Dividend Amount: {amount:.2f}" for amount in dividend_amounts]
        
        fig.add_trace(go.Scatter(x=dividend_rollout_days, y=price_when_dividend, 
                                 mode='markers', marker=dict(symbol='circle-open-dot', size=10), 
                                 showlegend=False, name="{} Dividend".format(stock),
                                 hovertext=hover_text))
    
    fig.update_layout(
        title='Stock Price with Dividend Rollout',
        xaxis_title='Date',
        yaxis_title='Stock Price',
        plot_bgcolor='white',
        xaxis=dict(showgrid=False),
        yaxis=dict(showgrid=False),
        hovermode="x unified"
        #hoverlabel=dict(bgcolor='white', font=dict(color='black'))
    )
    
    return fig

In [92]:
plot_price(stock_prices['LUPIN.NS'],stock_dividends['LUPIN.NS'])

In [99]:
def plot_multi_price(df_price, df_dividend, figure=None):
    
    """input dataframes"""
    
    if figure is None:
        fig = go.Figure()
    else:
        fig = figure
    
    for stock in df_price.columns:
        fig = plot_price(df_price[stock], df_dividend[stock], figure=fig)
        
    return fig

In [100]:
plot_multi_price(stock_prices, stock_dividends)

In [113]:
def plot_single_price_comparison(price_data, dividend_data, figure=None, stock=None):
    
    if figure is None:
        fig = go.Figure()
    else:
        fig = figure
    
    price_data = 100*price_data/price_data[0] - 100
        
    fig = plot_price(price_data,dividend_data)
    fig.add_trace(go.Scatter(x=price_data.index, 
                             y=np.ones(len(price_data))*price_data[0], 
                             mode='lines', line=dict(width=2,dash='dash'), showlegend=False))
    return fig

In [114]:
plot_single_price_comparison(stock_prices['LUPIN.NS'], stock_dividends['LUPIN.NS'])